In [1]:
import json, os
# from config import *
# from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
    TextAnalyticsClient,
    ExtractiveSummaryAction,
     AbstractiveSummaryAction
) 
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

class Summarization():

    def __init__(self, transcripts):
        self.transcripts = transcripts
        self.with_diarisation_flag = False
        self.document = []
        self.text_to_append = ""

        ###### CREDS #######
        self.key = "0b938542471248fda3aab56075a251c1"
        self.endpoint = "https://proj-rnd-uae-et-language-001.cognitiveservices.azure.com/"
        

    def get_text(self):

        transcriptions = self.transcripts["transcript"]
        #text_to_summarise = ""
        text_to_summarise_ls = []
        
        for item in transcriptions:
            text = ""
            if self.with_diarisation_flag:
                # text_to_summarise = text_to_summarise + item["speaker"] + ": " + item["dialogue"] + "\n"
                text =  item["speaker"] + ": " + item["dialogue"] 
            else:
            #    text_to_summarise = text_to_summarise +  item["dialogue"] + "\n" 
                text =  item["dialogue"] 
            text_to_summarise_ls.append(text)

        self.document = text_to_summarise_ls
        self.document= [" ".join(text_to_summarise_ls)]
    def summarise_main(self):
        pass



    # Authenticate the client using your key and endpoint 
    def authenticate_client(self):
        ta_credential = AzureKeyCredential(self.key)
        text_analytics_client = TextAnalyticsClient(
                endpoint=self.endpoint, 
                credential=ta_credential)
        return text_analytics_client

    

    # Example method for summarizing text
    def sample_extractive_summarization(self, client):

        document = self.document
        poller = client.begin_analyze_actions(
            document,
            actions=[
                ExtractiveSummaryAction(max_sentence_count=4)
                #  AbstractiveSummaryAction(max_sentence_count = 4)
            ],
        )

        document_results = poller.result()
        for result in document_results:
            extract_summary_result = result[0]  # first document, first result
            if extract_summary_result.is_error:
                print("...Is an error with code '{}' and message '{}'".format(
                    extract_summary_result.code, extract_summary_result.message
                ))
            else:
                print("Summary extracted: \n{}".format(
                    " ".join([sentence.text for sentence in extract_summary_result.sentences]))
                )

    

    



# if __name__ == "__main__":
    # with open(file= "./data/analytics/samplecopy_1.json", mode= "r") as fp:
    #     transcripts = json.load(fp= fp)
    #C:\Users\GagandeepSingh\Desktop\PROJ-EGYPT\PROJ-EGYPT\data\analytics\analytics_output.json
with open("./data/analytics/analytics_output.json") as fp:
    data = json.load(fp)

transcripts = data["result_2"]["transcripts"]
obj = Summarization(transcripts= transcripts)
obj.get_text()
# client = obj.authenticate_client()
# obj.sample_extractive_summarization(client)

In [12]:
import re
import pathlib

s1 = "data\audio_analytics\sample_audio"
p = pathlib.PureWindowsPath(s1)
print(str(p.as_posix()))

# re.sub(r'\\', '/', s1)

dataudio_analytics/sample_audio


In [3]:
obj.document[0]

'Hello. How can I help Manoj from C Bank\'s loan loan department? Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan.  Yes absolutely Vikram ji, you will be given complete information about the loan. I need some information from you before giving information about Alone. May I know what type of car you want to buy? We are thinking of buying a car that I want to take on loan duration_to_play.  "What are the required documents? I needed a little information from you about what I needed. Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy.\' Do you want an SUV or do you want a hatchback or do you want to buy some other type of vehicle? Can you give me information about it? are now. Thinking of buying a hatchback and my budget is up to ₹10,00,000. Yes, exactly like Vikrant ji you. information given. You want to buy a hatchback car an

In [61]:
import requests

LANGUAGE_ENDPOINT = "https://proj-rnd-uae-et-language-001.cognitiveservices.azure.com/"
url = f"{LANGUAGE_ENDPOINT}/language/analyze-text/jobs?api-version=2023-04-01"
key = "0b938542471248fda3aab56075a251c1"
text = obj.document[0]
data = {
  "displayName": "Document ext Summarization Task Example",
  "analysisInput": {
    "documents": [
      {
        "id": "1",
        "language": "en",
        "text": text
      }
    ]
  },
  "tasks": [
    {
      "kind": "ExtractiveSummarization",
      "taskName": "Document Extractive Summarization Task 1",
      "parameters": {
        "sentenceCount": 6
      }
    }
  ]
}

headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": key
}

res = requests.post(url = url, headers= headers, json= data)

In [62]:
res.status_code

202

In [63]:
operation_location = res.headers["operation-location"]
operation_location

'https://proj-rnd-uae-et-language-001.cognitiveservices.azure.com/language/analyze-text/jobs/b837a62e-1a33-497c-b3b3-620138a532cd?api-version=2023-04-01'

In [64]:
job_id = operation_location.split("/")[-1].split("?")[0]
job_id

'b837a62e-1a33-497c-b3b3-620138a532cd'

In [65]:

url = f"{LANGUAGE_ENDPOINT}/language/analyze-text/jobs/{job_id}?api-version=2023-04-01"
res2 = requests.get(url = url, headers= headers)

In [66]:
extractive_summary_response = res2

In [67]:
res2.status_code

200

In [68]:
extractive_summary_response = json.loads(extractive_summary_response.text)
extractive_summary_response

{'jobId': 'b837a62e-1a33-497c-b3b3-620138a532cd',
 'lastUpdatedDateTime': '2024-02-15T12:30:06Z',
 'createdDateTime': '2024-02-15T12:30:04Z',
 'expirationDateTime': '2024-02-16T12:30:04Z',
 'status': 'succeeded',
 'errors': [],
 'displayName': 'Document ext Summarization Task Example',
 'tasks': {'completed': 1,
  'failed': 0,
  'inProgress': 0,
  'total': 1,
  'items': [{'kind': 'ExtractiveSummarizationLROResults',
    'taskName': 'Document Extractive Summarization Task 1',
    'lastUpdateDateTime': '2024-02-15T12:30:06.4804872Z',
    'status': 'succeeded',
    'results': {'documents': [{'id': '1',
       'sentences': [{'text': 'Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan.',
         'rankScore': 1.0,
         'offset': 64,
         'length': 87},
        {'text': "Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy.'",
 

In [69]:
extractive_summary_response.keys()

dict_keys(['jobId', 'lastUpdatedDateTime', 'createdDateTime', 'expirationDateTime', 'status', 'errors', 'displayName', 'tasks'])

In [70]:
def get_extractive_summary(extractive_summary_response):
    status = False
    if extractive_summary_response["status"].lower() == "succeeded":
        extractive_summary = extractive_summary_response["tasks"]
        final_summary = ""
        for item in extractive_summary["items"][0]["results"]["documents"][0]["sentences"]:
            final_summary += item["text"]
        status = True
    else:
        errors = "".join(extractive_summary_response["errors"])

    if status:
        return {"status": "success", "summary": final_summary}
    else:
        return {"status": "fail", "error": errors}


In [71]:
get_extractive_summary(extractive_summary_response)

{'status': 'success',
 'summary': "Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan.Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy.'You want to buy a hatchback car and your budget is eight to ₹ 10,00,000.If you want to know about the interest rate of the loan and the documents and courses required, then I would like to tell you what we are right now.Whatever your loan amount is approved, a monthly EMI will be set up according to whatever interest you have.When you apply for a loan, you will have to copy your Aadhaar card along with the application form."}

In [55]:
type("") == str

True

In [51]:
final_summary.split(".")

['Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan',
 'Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy',
 "'You want to buy a hatchback car and your budget is eight to ₹ 10,00,000",
 'If you want to know about the interest rate of the loan and the documents and courses required, then I would like to tell you what we are right now',
 'Whatever your loan amount is approved, a monthly EMI will be set up according to whatever interest you have',
 'When you apply for a loan, you will have to copy your Aadhaar card along with the application form',
 '']

In [76]:
import requests, time

LANGUAGE_ENDPOINT = "https://proj-rnd-uae-et-language-001.cognitiveservices.azure.com/"
url = f"{LANGUAGE_ENDPOINT}/language/analyze-text/jobs?api-version=2023-04-01"
key = "0b938542471248fda3aab56075a251c1"
text = obj.document[0]
data = {
  "displayName": "Document ext Summarization Task Example",
  "analysisInput": {
    "documents": [
      {
        "id": "1",
        "language": "en",
        "text": text
      }
    ]
  },
  "tasks": [
    {
      "kind": "ExtractiveSummarization",
      "taskName": "Document Extractive Summarization Task 1",
      "parameters": {
        "sentenceCount": 6
      }
    }
  ]
}

headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": key
}

res = requests.post(url = url, headers= headers, json= data)
#if res.status_code == 202:

operation_location = res.headers["operation-location"]
job_id = operation_location.split("/")[-1].split("?")[0]
url = f"{LANGUAGE_ENDPOINT}/language/analyze-text/jobs/{job_id}?api-version=2023-04-01"
time.sleep(3)
extractive_summary_response= requests.get(url = url, headers= headers)
# status 200
if extractive_summary_response.status_code == 200:
  extractive_summary_response = json.loads(extractive_summary_response.text)

  result = get_extractive_summary(extractive_summary_response)
else:
  error_msg =  f"status code : {extractive_summary_response.status_code}. Response : {extractive_summary_response.text}"
  result = {"status": "fail", "error":error_msg}




In [77]:
result

{'status': 'success',
 'summary': "Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan.Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy.'You want to buy a hatchback car and your budget is eight to ₹ 10,00,000.If you want to know about the interest rate of the loan and the documents and courses required, then I would like to tell you what we are right now.Whatever your loan amount is approved, a monthly EMI will be set up according to whatever interest you have.When you apply for a loan, you will have to copy your Aadhaar card along with the application form."}

In [58]:
res.status_code == 202

True

In [60]:
f"abd {res.status_code}"

'abd 202'

In [1]:
from utils import *

In [14]:
text = "hi hello how are you, Where is Dr. Smith. What is his health"
ar_text= "مرحبًا، كيف حالك"

In [3]:
save_wordcloud(text = ar_text, language= "ar")

In [7]:
with open("./data/analytics/analytics_output.json") as fp:
        data = json.load(fp)

transcripts = data["result_2"]["transcripts"]
transcripts

{'transcript': [{'dialogue': 'Hello.',
   'speaker': 'Agent',
   'duration_to_play': '4.24',
   'sentiment': 'Neutral'},
  {'dialogue': "How can I help Manoj from C Bank's loan loan department?",
   'speaker': 'Agent',
   'duration_to_play': '5.05',
   'sentiment': 'Neutral'},
  {'dialogue': 'Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan. ',
   'speaker': 'Customer',
   'duration_to_play': '10.76',
   'sentiment': 'Neutral'},
  {'dialogue': 'Yes absolutely Vikram ji, you will be given complete information about the loan. I need some information from you before giving information about Alone. May I know what type of car you want to buy?',
   'speaker': 'Agent',
   'duration_to_play': '20.03',
   'sentiment': 'Positive'},
  {'dialogue': 'We are thinking of buying a car that I want to take on loan duration_to_play.  "What are the required documents?',
   'speaker': 'Customer',
   'duration_to_play': '45.05',
   'sentiment': 'Neutral'},
  {'dialogue

In [10]:
for item in transcripts["transcript"]:
    print(item["dialogue"])

Hello.
How can I help Manoj from C Bank's loan loan department?
Yeh Namaskar Manoj, my name is Vikrant and I wanted information about the vehicle loan. 
Yes absolutely Vikram ji, you will be given complete information about the loan. I need some information from you before giving information about Alone. May I know what type of car you want to buy?
We are thinking of buying a car that I want to take on loan duration_to_play.  "What are the required documents?
I needed a little information from you about what I needed.
Yes absolutely Vikrant ji, we have multiple plans for vehicle loans and before that I would like to know your budget and I would like to know what type of car you want to buy.' Do you want an SUV or do you want a hatchback or do you want to buy some other type of vehicle? Can you give me information about it?
are now.
Thinking of buying a hatchback and my budget is up to ₹10,00,000.
Yes, exactly like Vikrant ji you.
information given. You want to buy a hatchback car and y

In [13]:
topics = "\u062a\u062c\u062f\u064a\u062f \u0628\u0627\u0642\u0629 \u0627\u0644\u0625\u0646\u062a\u0631\u0646\u062a 75 \u060c \u0627\u0644\u0628\u0627\u0642\u0627\u062a \u0627\u0644\u0645\u062a\u0627\u062d\u0629 \u060c \u0627\u0644\u0639\u0631\u0648\u0636 \u0639\u0644\u0649 \u0627\u0644\u0628\u0627\u0642\u0627\u062a."

In [14]:
topics

'تجديد باقة الإنترنت 75 ، الباقات المتاحة ، العروض على الباقات.'

In [24]:
t = topics.split("،")
t

['تجديد باقة الإنترنت 75 ', ' الباقات المتاحة ', ' العروض على الباقات.']

In [20]:
with open(file= "./data/analytics/samplecopy_1.json", mode= "r") as fp:
        d = json.load(fp= fp)

In [21]:
d

{'result': {'summary': 'يبدو أن المحادثة بين عميل ووكيل فيما يتعلق بتجديد حزمة الإنترنت. يريد العميل تجديد باقة الإنترنت 75 ويقترح الوكيل باقة الإنترنت 70. يسأل العميل عن الشروط ويطلب الوكيل لحظة للتحقق. يسأل العميل عما إذا كان بحاجة إلى الذهاب إلى المقر الرئيسي ويقترح الوكيل القيام بذلك عبر الهاتف. يسأل العميل عن أي عروض ويقول الوكيل إنه لا يوجد أي منها متاح حاليا. يقرر العميل التجديد بدون عرض',
  'topics': [],
  'wordcloud': '...',
  'transcripts': {'transcript': [{'dialogue': 'صباح الخير، اتصالات بسم مع حضرتك.',
     'speaker': 'Agent',
     'duration_to_play': '0.6',
     'sentiment': 'Neutral'},
    {'dialogue': 'ايمن نور أحمد مع حضرتك؟',
     'speaker': 'Customer',
     'duration_to_play': '3.',
     'sentiment': 'Neutral'},
    {'dialogue': 'أهلا بيك يا أستاذ أحمد، إزيك عامل إيه يا فندم؟',
     'speaker': 'Agent',
     'duration_to_play': '4.5',
     'sentiment': 'Positive'},
    {'dialogue': 'أنا عايزة أجدد باقة الإنترنت 75.',
     'speaker': 'Customer',
     'duration_to_play'

In [25]:
d["result"]["topics"] = t

In [26]:
d["result"]["topics"]


['تجديد باقة الإنترنت 75 ', ' الباقات المتاحة ', ' العروض على الباقات.']

In [28]:
with open(file= "./data/analytics/samplecopy_2.json", mode= "w") as fp:
        json.dump(d,fp= fp, indent= 4)

In [3]:
import json
with open("./data/transcript/transcript_output.json") as fp:
    d1= json.load(fp)
d1


{'transcript': [{'dialogue': 'नमस्कार।',
   'speaker': 'Agent',
   'duration_to_play': '4.24',
   'locale': 'hi-IN'},
  {'dialogue': 'सी बैंक के लोन लोन डिपार्टमेंट से मैं मनोज आपकी किस प्रकार से सहायता कर सकता हूँ?',
   'speaker': 'Agent',
   'duration_to_play': '5.05',
   'locale': 'hi-IN'},
  {'dialogue': 'येह नमस्कार मनोज मेरा नाम विक्रांत है और मेरे को व्हीकल लोन के बारे में जानकारी चाहिए थी।',
   'speaker': 'Customer',
   'duration_to_play': '10.76',
   'locale': 'hi-IN'},
  {'dialogue': 'जी बिलकुल विक्रम जी, आपको लोन के बारे में पूरी जानकारी दी जाएगी। अलोन के बारे में जानकारी देने से पहले मुझे आपसे कुछ आप चेक इन्फॉर्मेशन की आवश्यकता है। क्या मैं जान सकता हूँ आप किस प्रकार की गाड़ी खरीदना चाहते हैं?',
   'speaker': 'Agent',
   'duration_to_play': '20.03',
   'locale': 'hi-IN'},
  {'dialogue': 'हम एक कार खरीदने की सोच रहा हूँ जो की मैं लोन पर लेना चाहता हूँ तो मुझे उसके बारे में की क्या उसकी इन्ट्रेस्ट?',
   'speaker': 'Customer',
   'duration_to_play': '34.92',
   'locale': 'hi-IN

In [17]:
ar_text

'مرحبًا، كيف حالك'

In [18]:
ar_text.lower()

'مرحبًا، كيف حالك'

In [19]:
s1 = "\u0928\u092e\u0938\u094d\u0915\u093e\u0930\u0964"
print(s1)

नमस्कार।


In [20]:
s2 = "\u0938\u0940 \u092c\u0948\u0902\u0915 \u0915\u0947 \u0932\u094b\u0928 \u0932\u094b\u0928 \u0921\u093f\u092a\u093e\u0930\u094d\u091f\u092e\u0947\u0902\u091f \u0938\u0947 \u092e\u0948\u0902 \u092e\u0928\u094b\u091c \u0906\u092a\u0915\u0940 \u0915\u093f\u0938 \u092a\u094d\u0930\u0915\u093e\u0930 \u0938\u0947 \u0938\u0939\u093e\u092f\u0924\u093e \u0915\u0930 \u0938\u0915\u0924\u093e \u0939\u0942\u0901?"
s2

'सी बैंक के लोन लोन डिपार्टमेंट से मैं मनोज आपकी किस प्रकार से सहायता कर सकता हूँ?'

In [21]:
a = [[],[],[]]
len(a)

3

In [4]:
"audio_01.wav".split(".")[:-1]

['audio_01']

In [25]:
from utils import *

path = LocalConfig().PROCESSED_DATA_FOLDER
for audio_file in os.listdir(path):
    # checks
    print(f"audio file : {audio_file}")
    extension = audio_file.split(".")[-1]
    filename = audio_file.split(".")[:-1]

    path1 = path + "/" + audio_file
    attrs = get_audio_attributes(path= path1)
    print(f"sample rate  {attrs.samplerate}")
    print(f"subtype : {attrs.subtype}")
    print(f"channels : {attrs.channels}")
    print()

audio file : Call 1.wav
sample rate  16000
subtype : PCM_16
channels : 1

audio file : Call 2.wav
sample rate  16000
subtype : PCM_16
channels : 1



In [4]:
import soundfile

soundfile._default_subtypes

{'WAV': 'PCM_16',
 'AIFF': 'PCM_16',
 'AU': 'PCM_16',
 'PAF': 'PCM_16',
 'SVX': 'PCM_16',
 'NIST': 'PCM_16',
 'VOC': 'PCM_16',
 'IRCAM': 'PCM_16',
 'W64': 'PCM_16',
 'MAT4': 'DOUBLE',
 'MAT5': 'DOUBLE',
 'PVF': 'PCM_16',
 'XI': 'DPCM_16',
 'HTK': 'PCM_16',
 'SDS': 'PCM_16',
 'AVR': 'PCM_16',
 'WAVEX': 'PCM_16',
 'SD2': 'PCM_16',
 'FLAC': 'PCM_16',
 'CAF': 'PCM_16',
 'WVE': 'ALAW',
 'OGG': 'VORBIS',
 'MPC2K': 'PCM_16',
 'RF64': 'PCM_16',
 'MP3': 'MPEG_LAYER_III'}

In [10]:
import uuid

str(uuid.uuid1())

'c48d9c40-d007-11ee-a6f1-8c16454a336a'

In [1]:
next_call_number = 1
'Call_{}'.format(next_call_number) + ".wav"

'Call_1.wav'

In [10]:
import json
# Use merged_output.json & audios_info/mappings.json
path1 = "data/audio_analytics/sample_audio/merged_output.json"
with open(path1) as fh:
    data1 = json.load(fh)

In [14]:
transcriptions = data1["result"]["transcripts"]["transcript"]
transcriptions[::-1]

[{'dialogue': 'धन्यवाद मैडम जी आपका भी।',
  'speaker': 'Customer',
  'duration_to_play': '311.23',
  'locale': 'hi-IN',
  'sentiment': 'negative'},
 {'dialogue': 'जी विकास जी, एलडीसी लोन डिपार्टमेंट में कॉल करने के लिए धन्यवाद। आपका दिन शुभ रहे।',
  'speaker': 'Agent',
  'duration_to_play': '306.18',
  'locale': 'hi-IN',
  'sentiment': 'neutral'},
 {'dialogue': 'जी विक्रम जी, क्या आप कोई और जानकारी प्राप्त करना चाहेंगे?',
  'speaker': 'Agent',
  'duration_to_play': '301',
  'locale': 'hi-IN',
  'sentiment': 'negative'},
 {'dialogue': 'की आपने मुझे ये जानकारी प्रोवाइड कराई जी धन्यवाद बहुत बहुत धन्यवाद आपका।',
  'speaker': 'Customer',
  'duration_to_play': '295.12',
  'locale': 'hi-IN',
  'sentiment': 'neutral'},
 {'dialogue': 'तो धन्यवाद धन्यवाद मनोज।',
  'speaker': 'Customer',
  'duration_to_play': '290.83',
  'locale': 'hi-IN',
  'sentiment': 'neutral'},
 {'dialogue': 'ठीक है आई थिंक ही मतलब मैं सोचता हूँ की मेरे को जो जानकारी चाहिए थी वो मिल गयी है।',
  'speaker': 'Customer',
  'dura

In [15]:
import pandas as pd
import json
with open("data/audios_info/mappings.json") as fh:
    data = json.load(fh)


In [19]:
dff = pd.DataFrame.from_dict(data)
dff

,Call 1.wav,Call 2.wav
id,Call_1.wav,Call_2.wav
audio_duration,82,107
audio_file_size,2563.917969,2563.917969
language,hindi,hindi
overall_sentiment,negative,negative
call_opening_score,0.0,0.0
call_closing_score,0.0,0.0


In [20]:
dff.columns

Index(['Call 1.wav', 'Call 2.wav'], dtype='object')

In [21]:
dff.index

Index(['id', 'audio_duration', 'audio_file_size', 'language',
       'overall_sentiment', 'call_opening_score', 'call_closing_score'],
      dtype='object')

In [22]:
output = []
for k,v in data.items():
    res = {}
    res["audio_filename"] = k
    for attr, value in v.items():
        res[attr] = value
    output.append(res)


In [23]:
dff = pd.DataFrame.from_dict(output)
dff

,audio_filename,id,audio_duration,audio_file_size,language,overall_sentiment,call_opening_score,call_closing_score
0,Call 1.wav,Call_1.wav,82,2563.917969,hindi,negative,0.0,0.0
1,Call 2.wav,Call_2.wav,107,2563.917969,hindi,negative,0.0,0.0


In [24]:
dff = pd.read_excel("PowerBi.xlsx")
if 

,Unnamed: 0,parts,duration,keywords,sentiment,dialouge
0,0,1,79.83,key phrase5,positive,नमस्कार।
1,1,1,79.83,key phrase7,positive,नमस्कार।
2,2,1,79.83,key phrase6,negative,सी बैंक के लोन लोन डिपार्टमेंट से मैं मनोज आपक...
3,3,1,79.83,key phrase7,negative,सी बैंक के लोन लोन डिपार्टमेंट से मैं मनोज आपक...
4,4,1,79.83,key phrase4,negative,येह नमस्कार मनोज मेरा नाम विक्रांत है और मेरे ...
...,...,...,...,...,...,...
89,89,5,311.23,key phrase2,negative,"जी विक्रम जी, क्या आप कोई और जानकारी प्राप्त क..."
90,90,5,311.23,key phrase4,neutral,"जी विकास जी, एलडीसी लोन डिपार्टमेंट में कॉल कर..."
91,91,5,311.23,key phrase2,neutral,"जी विकास जी, एलडीसी लोन डिपार्टमेंट में कॉल कर..."
92,92,5,311.23,key phrase1,negative,धन्यवाद मैडम जी आपका भी।


In [3]:
d1 = {
    "Call 1.wav": {
        "id": "Call_1.wav",
        "audio_duration": 82,
        "audio_file_size": 2563.91796875,
        "a":0
    },
    "Call 2.wav": {
        "id": "Call_2.wav",
        "audio_duration": 107,
        "audio_file_size": 2563.91796875
    }
}

In [4]:
len(d1)

2